# Préparation Deep Learning Images

In [44]:
import cv2
import glob
import numpy as np
import pandas as pd

import os
import joblib
from joblib import dump, load

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


# Fonctions

In [48]:

def path_file(i,df):
    ### fonction pour aller chercher le chemin d'accès d'un photo de rang i dans le dateframe train_image###
    img = df['File'].iloc[i]
    path_file = f"C:\\Users\\xavie\\Projet Rakuten\\images\\images\\image_train\\{img}"
    #path_file = f"C:\\Users\\Frederic\\Projet Rakuten\\images\\images\\image_train\\{img}"
    
    return path_file

def supprimer_bords(img, threshold_black=10, threshold_white=245):
    ### fonction pour supprimer les bords noirs ou blances des photos ###    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Pour les bords noirs
    _, thresh_black = cv2.threshold(gray, threshold_black, 255, cv2.THRESH_BINARY)
    contours_black, _ = cv2.findContours(thresh_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours_black) > 0:
        cnt_black = max(contours_black, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt_black)
        img = img[y:y+h, x:x+w]

    # Pour les bords blancs
    _, thresh_white = cv2.threshold(gray, threshold_white, 255, cv2.THRESH_BINARY_INV)
    contours_white, _ = cv2.findContours(thresh_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours_white) > 0:
        cnt_white = max(contours_white, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(cnt_white)
        img = img[y:y+h, x:x+w]

    return img


def traiter_images(filepath, IMG_SIZE, output_directory, echantillon=None):
    ### Fonction pour traiter les images du dataset et créer des repertoires pour le DL
    
    data = []  # Pour stocker les données (images traitées)
    
    # Utiliser glob pour obtenir la liste des fichiers correspondant au motif
    image_files = glob.glob(filepath + '/*.jpg')  
     
    # Initialiser un compteur pour suivre le nombre d'images traitées
    images_traitees = 0  
    
    # Liste pour stocker les exceptions
    exceptions = []  
            
    for image_file in image_files:
        if (echantillon is not None) and (images_traitees >= echantillon):
            break
        try:
            img = cv2.imread(image_file)
                        
            #traiter l'image
            image_resultat = supprimer_bords(img)         
            image_resultat = cv2.resize(image_resultat, (IMG_SIZE,IMG_SIZE))

            if image_resultat is not None:
                image_name = image_file.split('\\')[-1]  #obtenir le nom de l'image
                                                 # Enregistrer l'image résultante dans le dossier de sortie            
                output_path = os.path.join(output_directory, image_name)
                cv2.imwrite(output_path, image_resultat)
            print(output_path)  
            
            # Utiliser des opérations de chaîne pour extraire les informations nécessaires
            parts = image_name.split('_')
            
            image_id = parts[1]
            product_id = parts[3].split('.')[0]
            
            # Obtenez la largeur, la hauteur et la taille en bits de l'image
            height, width, size = image_resultat.shape[0], image_resultat.shape[1], image_resultat.nbytes*8
            
            # Créez un dictionnaire avec les informations extraites
            dict_image = {'File':image_name, 'image_id':image_id, 'product_id':product_id,'Height':height,
                        'Width': width, 'Size in Bits': size}
           
            # Ajouter le dictionnaire à la liste de données
            data.append(dict_image)

            # Incrémenter le compteur d'images traitées
            images_traitees += 1

        except FileNotFoundError:
            exceptions.append(f"Image {image_name} introuvable.")
        except Exception as e:
             exceptions.append(f"Erreur lors du traitement de l'image {image_file}: {str(e)}")

         
       # Affichez les exceptions à la fin
    if exceptions:
        print("\nExceptions rencontrées :")
    for exception in exceptions:
        print(exception)
    else:
        print("\nAucune exception rencontrée.")

    # Convertir la liste de données en un DataFrame
    df = pd.DataFrame(data)

    return df


# Traitement des images et création du Fd

In [49]:
# Réglage des paramètres

IMG_SIZE= 224
path_file_images_a_traiter = "C:\\Users\\xavier\\Projet Rakuten\\images\\images\\image_train"
path_file_images_traitees = "C:\\Users\\xavier\\Projet Rakuten\\images\\images\\images_traitees"

df_train_image=traiter_images(filepath=path_file_images_a_traiter, IMG_SIZE = IMG_SIZE,
                              output_directory = path_file_images_traitees)


Aucune exception rencontrée.


## Exploration sur les images

In [52]:
df_train_image = load('df_train_image.joblib')

#fonction pour aller chercher le chemin d'accès d'un photo de rang i dans les images d'entrainement
def path_file_2(i):
    img = df_train_image['File'].iloc[i]
    path_file = f"C:\\Users\Frederic\Projet Rakuten\images\images\image_train\{img}"
    return path_file


FileNotFoundError: [Errno 2] No such file or directory: 'df_train_image.joblib'

In [ ]:

# Affichage 4 images du dataset au hasard
images=[]
categories=[]
#Representation graphique
for k in np.random.choice(np.arange(0, len(df_train_image)), size = 4):
        lien = df_train_image['File'].iloc[k]
        path_file = f"C:\\Users\Frederic\Projet Rakuten\images\images\image_train\{lien}"
        img_color = cv2.imread(path_file)    
        images.append(img_color)
        categories.append(df_train_image['categorie'].iloc[k])

# Display some augmented samples
plt.figure(figsize=(9,9))
for count, value in enumerate(images):
    plt.subplot(2,2,count+1)
    plt.axis('off')
    plt.imshow(value+1)
    #plt.xticks([])
    #plt.yticks([])
    #plt.tight_layout()
    plt.title('Catégorie: ' + str(categories[count]))
plt.show()


In [43]:
# Autre option
# Affichage 4 images du dataset au hasard
images=[]
categories=[]

#Representation graphique
for k in np.random.choice(np.arange(0, len(df_train_image)), size = 4):
        lien = df_train_image['File'].iloc[k]
        path_file = f"C:\\Users\Frederic\Projet Rakuten\images\images\image_train\{lien}"
        img_color = cv2.imread(path_file)    
        images.append(img_color)
        categories.append(df_train_image['categorie'].iloc[k])

for j in [1,2,3,4]:
    fig = plt.figure(figsize=(9,9))
    ax = fig.add_subplot(2,2,j)
    ax.set_axis_off()
    ax.imshow(images[j-1]) #pour lire  mon_image 
    #Modification du titre de la figure
    plt.title('Catégorie: ' + str(categories[j-1]))
    plt.show();
    

# Affichage 4 images du dataset au hasard pour une catégorie 
categorie_choisie = 'livres'
df_categorie_choisie = df_train_image.loc[df_train_image["categorie"] == categorie_choisie]

images=[]

#Representation graphique
j= 1  
for k in np.random.choice(np.arange(0, len(df_categorie_choisie)), size = 4):
        lien = df_categorie_choisie['File'].iloc[k]
        path_file = f"C:\\Users\Frederic\Projet Rakuten\images\images\image_train\{lien}"
        img_color = cv2.imread(path_file)    
        images.append(img_color)

for j in [1,2,3,4]:
    fig = plt.figure(figsize=(9,9))
    ax = fig.add_subplot(2,2,j)
    ax.set_axis_off()
    ax.imshow(images[j-1]) #pour lire  mon_image 
    #Modification du titre de la figure
    plt.title('Catégorie: ' + str(categorie_choisie))
    plt.show();
        
    

ValueError: 'a' cannot be empty unless no samples are taken

# Chargement du DataFrame et Visualisation du traitement sur des images aléatoires du Dataset

In [ ]:
# Chargement du DataFrame
df_train_image = joblib.load("df_train_image.joblib") # Chargez votre propre DataFrame

j = 0
for i in np.random.choice(np.arange(0, len(df_train_image)), size = 3):
    j = j + 1
    img = cv2.imread(path_file(i,df_train_image))
          
    fig = plt.figure(figsize = (5,5))
    ax = fig.add_subplot(3,3,j)
           
    # Suppression des axes
    ax.set_axis_off()
    # Affichage de la figure 
    ax.imshow(img) #pour lire l'image non transformée
    # Titre de la figure
    plt.title('photo avant transformation')
    plt.show();  
    
    j = j + 1 
    cropped_image = supprimer_bords(i)
    
    fig = plt.figure(figsize = (5,5))
    ax = fig.add_subplot(3,3,j)
      
    # Suppression des axes
    ax.set_axis_off()
    # Affichage de la figure 
    ax.imshow(cropped_image) #pour lire l'image transformée
    #Titre de la figure
    plt.title('photo après transformation')
    plt.show();   
 

# Fusion du DataSet Image avec X_train et mise en forme

In [27]:
#Chargement et assemblage des autres Datasets

# PC perso
#y_train = pd.read_csv('Y_train_CVw08PX.csv',index_col=0)
#X_train_update = pd.read_csv("C:\\Users\Frederic\Projet Rakuten/X_train_update.csv",index_col=0)

# PC portable
path_X_train = "C:\\Users\\xavie\\Documents\\GitHub\\AVR23_CDS_Rakuten\\data\\X_train_update.csv"
path_y_train = "C:\\Users\\xavie\\Documents\\GitHub\\AVR23_CDS_Rakuten\\data\\Y_train_CVw08PX.csv"

X_train_update = pd.read_csv(path_X_train,index_col=0)   
y_train = pd.read_csv(path_y_train,index_col=0)

df = pd.concat([X_train_update,y_train], axis=1)

print(df.shape)

#Nommage de la cible
dictionnaire_cat={"10" : "livres", "40" : "jeux video", "50" : "accessoires jeux video", "60" : "console jeux video", "1140" : "goodies", "1160" : "cartes à collectionner" ,
                  "1180":"jeux de rôle", "1280" : "jouet", "1281": "jeux de societe", "1300" : "modélisme", "1301":"activité/jeux ext", "1302" : "activité plein air",
                  "1320": "puericulture" , "1560" : "equipement ameublement", "1920": "accessoire literie", "1940" : "confiserie", "2060" : "décoration", "2220" : "accessoire animaux",
                  "2280" : "magazine", "2403": "revue/album/serie", "2462" : "lot jeux video" ,"2522" : "fourniture bureau", "2582" : "equipement ext", "2583" : "accessoire piscine",
                  "2585" : "accessoire entretien ext", "2705": "livre documentaire", "2905" : "jeux vidéo dématérialisés"}

# Convertir la colonne "prdtypecode" en chaîne de caractères
df["prdtypecode"] = df["prdtypecode"].astype("str")
# Créer une nouvelle colonne "categorie" en utilisant le dictionnaire de catégories
df["categorie"] = df["prdtypecode"].map(dictionnaire_cat)

df_train_image = df_train_image.rename(columns={'product_id':'productid','image_id':'imageid'})

#retypage en str des colonnes   
df['productid'] = df['productid'].astype("str")
df["imageid"] = df["imageid"].astype("str")
df_train_image['productid'] = df_train_image['productid'].astype("str")
df_train_image['imageid'] = df_train_image['imageid'].astype("str")

#fusion des df
df_train_image = df_train_image.merge(right = df, how = 'inner', on = ["productid","imageid"])
to_drop=['designation','description']
df_train_image.drop(to_drop,axis=1) 
dump(df_train_image, 'df_train_image.joblib')

# creation ['df_train_image.joblib']

(84916, 5)


KeyError: 'productid'

In [ ]:
# Erreur à gérer

# Rangement des images modifiées  par Classe 

In [ ]:
# Séparation des données
X_train,X_test = train_test_split(df, test_size=0.2)

In [50]:
import os
import pandas as pd
import shutil

df = df_train_image

# Chemin vers le répertoire contenant les images à classer
image_dir = "C:\\Users\\xavie\\Projet Rakuten\\images\\images\\images_traitees"
    
# Chemin vers le répertoire de sortie où les images classées seront déplacées
output_dir = "C:\\Users\\xavie\\Projet Rakuten\\images\\images\\images_traitees_classe"

# Créez des sous-répertoires pour chaque classe
for class_code in df['prdtypecode'].unique():
    class_dir = os.path.join(output_dir, str(class_code))
    os.makedirs(class_dir, exist_ok=True)

    
# Train
# Liste pour stocker les exceptions
exceptions = []
# Parcourez le DataFrame et déplacez les images vers les sous-répertoires appropriés
for index, row in X_train.iterrows():
    image_name = row['File'].split('\\')[-1]
    productid = row['productid']
    imageid= row['imageid']
    class_code = row['prdtypecode']

    # Chemin complet de l'image source
    source_image_path = os.path.join(image_dir, image_name)

    # Chemin du répertoire de destination pour cette classe
    class_output_dir = os.path.join(output_dir_train, str(class_code))

    # Chemin complet de l'image de destination
    destination_image_path = os.path.join(class_output_dir, image_name)

    try:
        # Déplacez l'image vers le répertoire de destination
        shutil.move(source_image_path, destination_image_path)
        print(f"Image {image_name} classée avec succès.")
    except FileNotFoundError:
        exceptions.append(f"Image {image_name} introuvable.")
    except Exception as e:
        exceptions.append(f"Erreur lors du déplacement de l'image {image_name}: {str(e)}")

# Affichez les exceptions à la fin
if exceptions:
    print("\nExceptions rencontrées :")
    for exception in exceptions:
        print(exception)
else:
    print("\nAucune exception rencontrée.")

print("\nClassification automatique terminée.")

# Test
# Liste pour stocker les exceptions
exceptions = []
# Parcourez le DataFrame et déplacez les images vers les sous-répertoires appropriés

for index, row in X_test.iterrows():
    image_name = row['File'].split('\\')[-1]
    productid = row['productid']
    imageid= row['imageid']
    class_code = row['prdtypecode']

    # Chemin complet de l'image source
    source_image_path = os.path.join(image_dir, image_name)

    # Chemin du répertoire de destination pour cette classe
    class_output_dir = os.path.join(output_dir_test, str(class_code))

    # Chemin complet de l'image de destination
    destination_image_path = os.path.join(class_output_dir, image_name)

    try:
        # Déplacez l'image vers le répertoire de destination
        shutil.move(source_image_path, destination_image_path)
        print(f"Image {image_name} classée avec succès.")
    except FileNotFoundError:
        exceptions.append(f"Image {image_name} introuvable.")
    except Exception as e:
        exceptions.append(f"Erreur lors du déplacement de l'image {image_name}: {str(e)}")

# Affichez les exceptions à la fin
if exceptions:
    print("\nExceptions rencontrées :")
    for exception in exceptions:
        print(exception)
else:
    print("\nAucune exception rencontrée.")

print("\nClassification automatique terminée.")

KeyError: 'prdtypecode'

## Genération d'images

In [51]:
# Chargement du DataFrame

#Chemin Frédéric
path = "C:\\Users\\xavie\\Documents\\GitHub\\AVR23_CDS_Rakuten\\data\\df_train_image.joblib" 
#Chemin Frédéric Hanène
#Chemin Franck
#CHemin karima

df_train_image = joblib.load(path) 

# fonction pour le chemin dans le nouveau dataaset
def path_file_2(i):
    classe= str(df_train_image['prdtypecode'].iloc[i])
    print(classe)
    img = df_train_image['File'].iloc[i]
    
    #Chemin Frédéric
    path_file = f"C:\\Users\\Frederic\\Projet Rakuten\\images\\images\\train\\{classe}\{img}"
    
    print(path_file)
    return path_file

# Visualisation de la génération d'images pour une image du Dataset

numero_image= 266 # a renseigner

# Chargement de l'image et transformation en matrice et expansion des dimensions
img = load_img(path_file_2(numero_image))
img = img_to_array(img)
img1 = np.expand_dims(img, axis=0)
 
# Définir les paramètres du générateur d'images aléatoires
generator = ImageDataGenerator(
    #rescale=1.0 / 255.0, #rescale
    rotation_range=20,  # Angle de rotation aléatoire
    width_shift_range=0.2,  # Déplacement horizontal aléatoire
    height_shift_range=0.2,  # Déplacement vertical aléatoire
    shear_range=0.2,  # Déformation aléatoire
    zoom_range=0.2,  # Zoom aléatoire
    horizontal_flip=True,  # Retournement horizontal aléatoire
    fill_mode='nearest'  # Mode de remplissage pour les nouvelles zones créées
)

data_generator = generator.flow(img1,batch_size=2)
 
# Visualisation de quelques augmentations d'images
plt.figure(figsize=(10,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    for x in data_generator:
        plt.imshow(x[0]/255.)
        plt.xticks([])
        plt.yticks([])
        break
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\xavie\\Documents\\GitHub\\AVR23_CDS_Rakuten\\data\\df_train_image.joblib'